In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/openfood.csv")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   code                     1000 non-null   int64  
 1   product_name             1000 non-null   object 
 2   energy-kcal_100g         1000 non-null   float64
 3   energy_100g              1000 non-null   float64
 4   fat_100g                 1000 non-null   float64
 5   saturated-fat_100g       1000 non-null   float64
 6   trans-fat_100g           1000 non-null   float64
 7   cholesterol_100g         1000 non-null   float64
 8   carbohydrates_100g       1000 non-null   float64
 9   sugars_100g              1000 non-null   float64
 10  fiber_100g               1000 non-null   float64
 11  proteins_100g            1000 non-null   float64
 12  salt_100g                1000 non-null   float64
 13  sodium_100g              1000 non-null   float64
 14  calcium_100g             

In [2]:
# import findspark
# findspark.init()

import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local").appName("WordCountInline").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/20 13:12:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/05/20 13:12:41 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [77]:
spark.stop()

24/05/20 13:09:54 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/05/20 13:09:54 INFO SparkUI: Stopped Spark web UI at http://192.168.194.153:4040
24/05/20 13:09:54 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/05/20 13:09:54 INFO MemoryStore: MemoryStore cleared
24/05/20 13:09:54 INFO BlockManager: BlockManager stopped
24/05/20 13:09:54 INFO BlockManagerMaster: BlockManagerMaster stopped
24/05/20 13:09:54 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/05/20 13:09:54 INFO SparkContext: Successfully stopped SparkContext


In [4]:
import json

In [5]:
with open('../configs/features.json', "r") as columns_file:
    features = json.load(columns_file)

In [6]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql import types, functions

In [7]:
dataset: pyspark.sql.DataFrame = spark.read.csv('../data/openfood.csv', header=True, inferSchema=True)

In [13]:
spark.stop()

In [8]:
FEATURES_COLUMN = "scaled_feature"

id_columns = features["id"]
feature_numeric = features["numeric"]
numeric_columns = [
    functions.col(c).cast("float").alias(c) for c in feature_numeric
]
cat_columns = features["categorical"]

all_columns = id_columns + numeric_columns + cat_columns
df_with_selected_columns = dataset.select(*all_columns)

vec_assembler = VectorAssembler(
    inputCols=feature_numeric, outputCol="features"
)
df_with_features = vec_assembler.transform(df_with_selected_columns)

In [9]:
scaler = StandardScaler(inputCol="features", outputCol=FEATURES_COLUMN)

In [10]:
scaler_model = scaler.fit(df_with_features)

24/05/20 13:13:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
df_scaled_features = scaler_model.transform(df_with_features)

In [12]:
df_scaled_features.pandas_api()

/Users/a.gorbatovskiy/miniconda/envs/sparkenv/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


,code,product_name,energy-kcal_100g,energy_100g,fat_100g,saturated-fat_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,sodium_100g,calcium_100g,iron_100g,nutrition-score-fr_100g,categories_en,features,scaled_feature
0,99482455804,"365 everyday value, dark red kidney beans",85.000000,356.0,0.000000,0.000000,0.000,0.00000,15.380000,0.770000,5.400000,5.380000,0.020000,0.008000,0.015000,0.001380,-7.0,"Plant-based foods and beverages,Plant-based fo...","[85.0, 356.0, 0.0, 0.0, 0.0, 0.0, 15.380000114...","[0.4784856258824813, 0.4789684143108942, 0.0, ..."
1,7771712000765,Chocolates Surtidos con Frutas y Sabores,571.000000,2390.0,38.099998,21.400000,0.000,0.02380,47.599998,42.900002,0.000000,9.520000,0.179000,0.071400,0.238000,0.001900,26.0,"Snacks,Sweet snacks,Cocoa and its products,Con...","[571.0, 2390.0, 38.099998474121094, 21.3999996...","[3.214297557398786, 3.2155463769748236, 2.5448..."
2,79893065069,Sauce,167.000000,699.0,0.000000,0.000000,0.000,0.00000,45.000000,40.000000,1.700000,0.000000,0.000000,0.000000,0.000000,0.000000,7.0,"Plant-based foods and beverages,Plant-based fo...","(167.0, 699.0, 0.0, 0.0, 0.0, 0.0, 45.0, 40.0,...","(0.9400835237926397, 0.9404464089980761, 0.0, ..."
3,44500003864,Italian dry salame light less fat deli thin sl...,250.000000,1046.0,14.285714,5.357143,0.000,0.10700,3.571429,3.571429,0.000000,25.000000,4.464286,1.785714,0.071000,0.001290,18.0,"Meats and their products,Meats,Prepared meats,...","[250.0, 1046.0, 14.285714149475098, 5.35714292...","[1.407310664360239, 1.4073060712617846, 0.9542..."
4,71007373923,Beef barbacoa shredded beef in a zesty red sau...,153.000000,640.0,6.270000,1.760000,0.000,0.01400,17.650000,1.180000,0.400000,6.270000,1.295000,0.518000,0.059000,0.001060,4.0,Frozen foods,"[153.0, 640.0, 6.269999980926514, 1.7599999904...","[0.8612741265884664, 0.8610668122443043, 0.418..."
5,826766162116,Authentic italian pinwheels,236.000000,987.0,11.960000,5.070000,0.000,0.04200,17.030001,1.810000,0.700000,14.130000,1.902500,0.761000,0.091000,0.001960,15.0,Sandwiches,"[236.0, 987.0, 11.960000038146973, 5.070000171...","[1.3285012671560656, 1.3279264745080128, 0.798..."
6,757339777775,"Sticky fingers smokehouse, barbecue sauce, ten...",132.000000,552.0,0.000000,0.000000,0.000,0.00000,32.349998,26.469999,0.000000,0.000000,2.205000,0.882000,0.059000,0.001060,15.0,"Condiments,Sauces,Barbecue sauces,Groceries","[132.0, 552.0, 0.0, 0.0, 0.0, 0.0, 32.34999847...","[0.7430600307822063, 0.7426701255607124, 0.0, ..."
7,55742505696,Macaroni & Cheese,165.000000,689.0,5.880000,2.350000,0.118,0.00588,22.000000,3.140000,1.570000,6.270000,0.843000,0.337000,0.088200,0.000392,3.0,"Meals,Pasta dishes,Macaroni and cheese","[165.0, 689.0, 5.880000114440918, 2.3499999046...","[0.9288250384777578, 0.9269922400567587, 0.392..."
8,70221103118,Cote D'Or Experiences Chocolate Bar Dark 86% 1...,675.000000,2824.0,57.500000,35.000000,0.000,0.00000,32.500000,12.500000,12.500000,7.500000,0.000000,0.000000,0.058000,0.008700,15.0,"Snacks,Sweet snacks,Confectioneries","[675.0, 2824.0, 57.5, 35.0, 0.0, 0.0, 32.5, 12...","[3.7997387937726455, 3.7994573090279924, 3.840..."
9,71140660201,Organic coconut milk,185.000000,774.0,18.459999,16.920000,0.000,0.00000,4.620000,1.540000,0.000000,1.540000,0.077500,0.031000,0.000000,0.000000,10.0,"Plant-based foods and beverages,Beverages,Dair...","[185.0, 774.0, 18.459999084472656, 16.92000007...","[1.0414098916265768, 1.0413526760579555, 1.233..."


In [12]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_feature")

In [13]:
import pandas as pd

In [40]:
list(df.columns.values[2:-1])

['energy-kcal_100g',
 'energy_100g',
 'fat_100g',
 'saturated-fat_100g',
 'trans-fat_100g',
 'cholesterol_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fiber_100g',
 'proteins_100g',
 'salt_100g',
 'sodium_100g',
 'calcium_100g',
 'iron_100g',
 'nutrition-score-fr_100g']

In [26]:
df = pd.read_csv("../data/openfood.csv")

In [30]:
df_with_features.pandas_api()

24/05/20 12:54:46 ERROR Executor: Exception in task 0.0 in stage 11.0 (TID 11)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda/0x000000012fea07e0`: (struct<energy-kcal_100g:double,energy_100g:double,fat_100g:double,saturated-fat_100g:double,trans-fat_100g:double,cholesterol_100g:double,carbohydrates_100g:double,sugars_100g:double,fiber_100g:double,proteins_100g:double,salt_100g:double,sodium_100g:double,calcium_100g:double,iron_100g:double,nutrition-score-fr_100g:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.ap

Py4JJavaError: An error occurred while calling o1005.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11) (192.168.194.153 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda/0x000000012fea07e0`: (struct<energy-kcal_100g:double,energy_100g:double,fat_100g:double,saturated-fat_100g:double,trans-fat_100g:double,cholesterol_100g:double,carbohydrates_100g:double,sugars_100g:double,fiber_100g:double,proteins_100g:double,salt_100g:double,sodium_100g:double,calcium_100g:double,iron_100g:double,nutrition-score-fr_100g:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExe

24/05/20 12:54:46 ERROR Executor: Exception in task 0.0 in stage 12.0 (TID 12)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda/0x000000012fea07e0`: (struct<energy-kcal_100g:double,energy_100g:double,fat_100g:double,saturated-fat_100g:double,trans-fat_100g:double,cholesterol_100g:double,carbohydrates_100g:double,sugars_100g:double,fiber_100g:double,proteins_100g:double,salt_100g:double,sodium_100g:double,calcium_100g:double,iron_100g:double,nutrition-score-fr_100g:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.ap

Py4JJavaError: An error occurred while calling o1050.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 12.0 failed 1 times, most recent failure: Lost task 0.0 in stage 12.0 (TID 12) (192.168.194.153 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda/0x000000012fea07e0`: (struct<energy-kcal_100g:double,energy_100g:double,fat_100g:double,saturated-fat_100g:double,trans-fat_100g:double,cholesterol_100g:double,carbohydrates_100g:double,sugars_100g:double,fiber_100g:double,proteins_100g:double,salt_100g:double,sodium_100g:double,calcium_100g:double,iron_100g:double,nutrition-score-fr_100g:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExe

In [31]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_feature")
scaler_model = scaler.fit(df_with_features)
df_scaled_features = scaler_model.transform(df_with_features)

24/05/20 12:55:05 ERROR Executor: Exception in task 0.0 in stage 13.0 (TID 13)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda/0x000000012fea07e0`: (struct<energy-kcal_100g:double,energy_100g:double,fat_100g:double,saturated-fat_100g:double,trans-fat_100g:double,cholesterol_100g:double,carbohydrates_100g:double,sugars_100g:double,fiber_100g:double,proteins_100g:double,salt_100g:double,sodium_100g:double,calcium_100g:double,iron_100g:double,nutrition-score-fr_100g:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.ap

Py4JJavaError: An error occurred while calling o1056.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 13.0 failed 1 times, most recent failure: Lost task 0.0 in stage 13.0 (TID 13) (192.168.194.153 executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`VectorAssembler$$Lambda/0x000000012fea07e0`: (struct<energy-kcal_100g:double,energy_100g:double,fat_100g:double,saturated-fat_100g:double,trans-fat_100g:double,cholesterol_100g:double,carbohydrates_100g:double,sugars_100g:double,fiber_100g:double,proteins_100g:double,salt_100g:double,sodium_100g:double,calcium_100g:double,iron_100g:double,nutrition-score-fr_100g:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefi

In [8]:
vec_assembler = VectorAssembler(
    inputCols=feature_column_names, outputCol="features"
)

df_with_features = vec_assembler.transform(df_with_selected_columns)

In [9]:
df_with_features = vec_assembler.transform(df_with_selected_columns)

24/05/20 12:46:01 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [28]:
dataset['energy-kcal_100g']

Column<'energy-kcal_100g'>